<a href="https://colab.research.google.com/github/vamshap/PySpark-Challenges/blob/main/OddEvenMeasurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import col,when,to_date,sum

from datetime import datetime

# write a pyspark code to calculate the sum of measurements for each day and for all odd and even
# Initialize Spark Session
spark = SparkSession.builder.appName("MeasurementsTable").getOrCreate()

# Define the schema for the DataFrame
schema = StructType([
    StructField("measurement_id", IntegerType(), True),
    StructField("measurement_value", FloatType(), True),
    StructField("measurement_time", TimestampType(), True)
])

# Data to insert into the DataFrame
data = [
    (131233, 1109.51, datetime.strptime('07/10/2022 09:00:00', '%m/%d/%Y %H:%M:%S')),
    (135211, 1662.74, datetime.strptime('07/10/2022 11:00:00', '%m/%d/%Y %H:%M:%S')),
    (523542, 1246.24, datetime.strptime('07/10/2022 13:15:00', '%m/%d/%Y %H:%M:%S')),
    (143562, 1124.50, datetime.strptime('07/11/2022 15:00:00', '%m/%d/%Y %H:%M:%S')),
    (346462, 1234.14, datetime.strptime('07/11/2022 16:45:00', '%m/%d/%Y %H:%M:%S'))
]

# Create the DataFrame
measurements_df = spark.createDataFrame(data, schema=schema)

# Show the DataFrame
oddEven_df = measurements_df.withColumn("oddorEven",when (col("measurement_id")%2 == 0,"Even").when(col("measurement_id")%2 == 1,"Odd").otherwise("not"))
oddEven_df.show()
measurementValue_df  = oddEven_df.groupBy("oddorEven",to_date(col('measurement_time')).alias("date_time")).agg(sum(col("measurement_value")).alias("oddorEvenValue"))

measurementValue_df.show()




+--------------+-----------------+-------------------+---------+
|measurement_id|measurement_value|   measurement_time|oddorEven|
+--------------+-----------------+-------------------+---------+
|        131233|          1109.51|2022-07-10 09:00:00|      Odd|
|        135211|          1662.74|2022-07-10 11:00:00|      Odd|
|        523542|          1246.24|2022-07-10 13:15:00|     Even|
|        143562|           1124.5|2022-07-11 15:00:00|     Even|
|        346462|          1234.14|2022-07-11 16:45:00|     Even|
+--------------+-----------------+-------------------+---------+

+---------+----------+------------------+
|oddorEven| date_time|    oddorEvenValue|
+---------+----------+------------------+
|      Odd|2022-07-10|           2772.25|
|     Even|2022-07-11|2358.6400146484375|
|     Even|2022-07-10| 1246.239990234375|
+---------+----------+------------------+

